In [2]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, diags, spdiags
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt

In [5]:
def laplacian_eigenmaps(X, k=2, t=10, sigma=0.3):
    # Efficient nearest neighbors search
    nbrs = NearestNeighbors(n_neighbors=t + 1).fit(X)
    distances, indices = nbrs.kneighbors(X)

    m = X.shape[0]

    # Efficient weight matrix calculation
    S = np.exp(-distances ** 2 / (sigma ** 2))
    W = np.zeros((m, m))
    for i in range(m):
        W[i, indices[i, 1:]] = S[i, 1:]

    # Graph Laplacian
    D = diags(np.sum(W, axis=1))
    L = D - W

    # Eigenvalue and eigenvector computation using sparse matrix
    num_eigenvalues = k + 1  # +1 for the trivial eigenvector
    eigenvalues, eigenvectors = eigsh(L, k=num_eigenvalues, which='SM')
    
    # Sort eigenvalues and eigenvectors
    sorted_indices = np.argsort(eigenvalues)
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Project data onto the top k eigenvectors
    reduced_data = eigenvectors[:, 1:k+1]

    return reduced_data

In [7]:
def avg_laplacian_eigenmaps(X, k=2, t=10):
    # Efficient nearest neighbors search
    nbrs = NearestNeighbors(n_neighbors=t + 1).fit(X)
    distances, indices = nbrs.kneighbors(X)

    m = X.shape[0]
    # Initialize an "average distance matrix" A
    A = np.zeros((m, 1))

    for i in range(m):
        sum_of_squares = np.sum(np.square(distances[i, 1:]))
        A[i, 0] = np.sqrt(sum_of_squares) / t

    # Efficient weight matrix calculation
    S = -distances ** 2
    W = np.zeros((m, m))
    for i in range(m):
        row = t * A[i] * np.squeeze(A[indices[i, 1:]])
        W[i, indices[i, 1:]] = np.exp(S[i, 1:] / row)

    # Graph Laplacian
    D = diags(np.sum(W, axis=1))
    L = D - W

    # Eigenvalue and eigenvector computation using sparse matrix
    num_eigenvalues = k + 1  # +1 for the trivial eigenvector
    eigenvalues, eigenvectors = eigsh(L, k=num_eigenvalues, which='SM')

    # Sort eigenvalues and eigenvectors
    sorted_indices = np.argsort(eigenvalues)
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Project data onto the top k eigenvectors
    reduced_data = eigenvectors[:, 1:k+1]

    return reduced_data

In [ ]:
def var_radius_laplacian_eigenmaps(X, k=2, max_rad=50):
    # Efficient max_rad nearest neighbors search
    nbrs = NearestNeighbors(n_neighbors=max_rad + 1).fit(X)
    distances, indices = nbrs.kneighbors(X)

    m = X.shape[0]
    # Initialize an "average distance matrix" A
    A = np.zeros((m, 1))

    # Calculate avg distances based on sample radius max_rad
    for i in range(m):
        sum_of_squares = np.sum(np.square(distances[i, 1:]))
        A[i, 0] = np.sqrt(sum_of_squares) / max_rad

    # Calculate average of the avg distance matrix
    avg_distance = np.mean(A)

    # Efficient weight matrix calculation
    S = -distances ** 2
    W = np.zeros((m, m))
    for i in range(m):
        # smaller average distance for x_i necessitates larger radius
        cur_rad = int(min(max_rad * (avg_distance / A[i, 0]), max_rad))
        row = cur_rad * A[i, 0] * np.squeeze(A[indices[i, 1:cur_rad + 1]])
        W[i, indices[i, 1:cur_rad + 1]] = np.exp(S[i, 1:cur_rad + 1] / row)

    # Graph Laplacian
    D = diags(np.sum(W, axis=1))
    L = D - W

    # Eigenvalue and eigenvector computation using sparse matrix
    num_eigenvalues = k + 1  # +1 for the trivial eigenvector
    eigenvalues, eigenvectors = eigsh(L, k=num_eigenvalues, which='SM')

    # Sort eigenvalues and eigenvectors
    sorted_indices = np.argsort(eigenvalues)
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Project data onto the top k eigenvectors
    reduced_data = eigenvectors[:, 1:k+1]

    return reduced_data

In [ ]:
def var_radius_std_laplacian_eigenmaps(X, k=2, max_rad=50, sigma=2):
    # Efficient max_rad nearest neighbors search
    nbrs = NearestNeighbors(n_neighbors=max_rad + 1).fit(X)
    distances, indices = nbrs.kneighbors(X)

    m = X.shape[0]
    # Initialize an "average distance matrix" A
    A = np.zeros((m, 1))

    # Calculate avg distances based on sample radius max_rad
    for i in range(m):
        sum_of_squares = np.sum(np.square(distances[i, 1:]))
        A[i, 0] = np.sqrt(sum_of_squares) / max_rad

    # Calculate average of the avg distance matrix
    avg_distance = np.mean(A)

    # Efficient weight matrix calculation
    S = np.exp(-distances ** 2 / (sigma ** 2))
    W = np.zeros((m, m))
    for i in range(m):
        # smaller average distance for x_i necessitates larger radius
        cur_rad = int(min(max_rad * (avg_distance / A[i, 0]), max_rad))
        W[i, indices[i, 1:cur_rad + 1]] = S[i, 1:cur_rad + 1]

    # Graph Laplacian
    D = diags(np.sum(W, axis=1))
    L = D - W

    # Eigenvalue and eigenvector computation using sparse matrix
    num_eigenvalues = k + 1  # +1 for the trivial eigenvector
    eigenvalues, eigenvectors = eigsh(L, k=num_eigenvalues, which='SM')

    # Sort eigenvalues and eigenvectors
    sorted_indices = np.argsort(eigenvalues)
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Project data onto the top k eigenvectors
    reduced_data = eigenvectors[:, 1:k+1]

    return reduced_data